In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
anime = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation_systems/anime_data/anime.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation_systems/anime_data/rating.csv/rating.csv')



In [5]:
anime['genre'] = anime['genre'].astype(str)
anime['main genre'] = anime['genre'].str.split(', ').str[0]


anime = anime[anime['rating'] != -1]
ratings = ratings[ratings['rating'] != -1]

anime = anime.drop(list(anime.loc[anime['rating'].isnull() == True].index), axis=0) ##.index and axis = 0  to ensure it only works on the one column
anime = anime.fillna('Unknown') ## replaces any blanks in non-numeric data
anime =  anime.reset_index(drop = True)

In [7]:
tfidf = TfidfVectorizer(stop_words='english')
genres_matrix = tfidf.fit_transform(anime['genre'])

genres_matrix.shape
## looks at the different words in the strings for future comparison

(12064, 47)

In [8]:
## now use cosin similarity to find the number of similarities between them
similar_genres = linear_kernel(genres_matrix, genres_matrix)
similar_genres

array([[1.        , 0.14720571, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.14720571, 1.        , 0.17888377, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.17888377, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ]])

In [9]:
## map the indexs
indices = pd.Series(anime.index, index=anime['name']).drop_duplicates()
indices

,0
name,
Kimi no Na wa.,0
Fullmetal Alchemist: Brotherhood,1
Gintama°,2
Steins;Gate,3
Gintama&#039;,4
...,...
Toushindai My Lover: Minami tai Mecha-Minami,12059
Under World,12060
Violence Gekiga David no Hoshi,12061


In [28]:
### rec based on genres

def get_recommendations(name, similar_genres):
    idx = indices[name]
    sim_score = list(enumerate(similar_genres[idx]))
    sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)
    sim_score = sim_score[1:10]
    anime_indices = [i[0] for i in sim_score]
    return(anime['name'].iloc[anime_indices])

get_recommendations('Death Note',similar_genres)

,name
778,Death Note Rewrite
981,Mousou Dairinin
144,Higurashi no Naku Koro ni Kai
1383,Higurashi no Naku Koro ni Rei
445,Mirai Nikki (TV)
4164,Mirai Nikki (TV): Ura Mirai Nikki
334,Higurashi no Naku Koro ni
38,Monster
5382,AD Police
